<a href="https://colab.research.google.com/github/mertserbes/wargame3d/blob/main/Wargame3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 4B'den büyük, resmi instruct model:
MODEL_NAME = "Qwen/Qwen2.5-14B-Instruct"
# İstersen Qwen3 kullanmak için sadece bu satırı değiştir:
# MODEL_NAME = "Qwen/Qwen3-14B"

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    device_map="auto",      # H100'de otomatik yerleştir
    torch_dtype="auto"      # genelde bf16/fp16
)

def generate_with_llm(prompt: str) -> str:
    """
    Qwen 14B (HF transformers) ile JSON üreten basit wrapper.
    """
    messages = [
        {
            "role": "system",
            "content": (
                "You are a military air combat planner. "
                "ALWAYS return ONLY valid JSON. No explanations, no markdown."
            )
        },
        {
            "role": "user",
            "content": prompt
        }
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_new_tokens=512,
            temperature=0.15,
            top_p=0.9
        )

    # Sadece yeni üretilen kısmı decode et
    generated_ids = output_ids[0][input_ids.shape[-1]:]
    text = tokenizer.decode(generated_ids, skip_special_tokens=True)
    return text.strip()
